
BUILDING & TRADING FX QUANTITATIVE MODELS WITH AI & SIGNAL FILTERING

0. INTRODUCING THE APPROACH

1. PREPARING THE DATABASE WITH CURRENCY INDEXES

2. FILTERING NOISE WITH A LOWPASS FILTER

3. PREPARING THE DEEP LEARNING MODEL (LSTM)

4. TRAINING THE NEURONS

5. VALIDATING THE TRADING SIGNALS

6. INCORPORING MONEY MANAGEMENT

7. BACKTESTING

8. INCORPORING RISK MANAGEMENT

9. PREPARING THE EXECUTION

10. RUNNING THE TRADING SYSTEM

11. MONITORING THE SYSTEM



0. INTRODUCING THE APPROACH

Foreign Exchange (FX) trading is one of the most competitive activities in the world because of its market efficiency (data randomnes) and high volatility. Besides other financial instruments, FX makes traditional portfolio managers title it as gambling because of its difficulty in price prediction. Most big firms use FX to cover their currency risk on the positions of international commodities, assets and other tradeables. Although its an excellent market for algorithmic trading - specially in the sell-side - for its 24-5 continuity, high transactional volume and excellent liquidity. Most trading systems fail after 2 to 4 years because of its approach based in linear models or traditional indicators in a chaotic universe. High Frequency Trading systems are beneficiated mostly by noise and not by information

Financial markets are continuetly evolving. I like to say that markets are the spectrogram of society. 

In this paper I'd like to demostrate that FX can be predicated 

To simplify it, we have two approaches when it comes to trading, follow the trend or mean-reversion strategies. However, the trend concept 

FX it's very corrective, however when a strong trend starts caused by any macroeconomic factor you better surf the trend and not fight against it if you want to survive.

In this notebook I'll expalin my fundamental approach to FX quantitative trading. First we will simplify 45 pairs to 10 currency indices, which will allow us to observe the market desegmented and catch easier the trending pairs and the corrective ones.

Through this notebook I'll explain step by step each process and its logic. Hope you enjoy it.

In [6]:
# 1. PREPARING THE DATABASE WITH CURRENCY INDEXES

# import libraries for the module
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
from datetime import timedelta, date, datetime
from pathlib import Path
from numpy import isnan
import pandas as pd
pd.options.plotting.backend = "plotly"


# 
TOKEN = "37d11dc6e01ec900277bcf70ed296912-60827d532c22d5b1b0d3583a0e9309ef" # YOUR OANDA TOKEN
#
YEAR = 2022 # YEARS TO START
#

# instruments universe
symbols = ['AUD_CAD', 'AUD_CHF', 'AUD_HKD', 'AUD_JPY', 'AUD_NZD', 'AUD_SGD', 'AUD_USD',
           'CAD_CHF', 'CAD_HKD', 'CAD_JPY', 'CAD_SGD', 'CHF_HKD', 'CHF_JPY', 'EUR_AUD',
           'EUR_CAD', 'EUR_CHF', 'EUR_GBP', 'EUR_HKD', 'EUR_JPY', 'EUR_NZD', 'EUR_SGD',
           'EUR_USD', 'GBP_AUD', 'GBP_CAD', 'GBP_CHF', 'GBP_HKD', 'GBP_JPY', 'GBP_NZD',
           'GBP_SGD', 'GBP_USD', 'HKD_JPY', 'NZD_CAD', 'NZD_CHF', 'NZD_HKD', 'NZD_JPY',
           'NZD_SGD', 'NZD_USD', 'SGD_CHF', 'SGD_HKD', 'SGD_JPY', 'USD_CAD', 'USD_CHF',
           'USD_HKD', 'USD_JPY', 'USD_SGD']  # 45 FX pairs


def importdb(year, symbols, make_indexes):

    api = API(access_token=TOKEN)

    # extract currencies (ccys)
    ccys = []
    if make_indexes:
        for sym in symbols:
            p = sym.split('_')
            if len(p[0]) == 3 and p[0] not in ccys:
                ccys.append(p[0])
            if len(p[1]) == 3 and p[1] not in ccys:
                ccys.append(p[1])
            ccys.sort()

    # prepare dataframe
    def prepdf(year):

        def daterange(date1, date2):
            for n in range(int((date2 - date1).days)+1):
                yield date1 + timedelta(n)

        start_dt = date(year, 1, 1)
        end_dt = date(year, 12, 31)

        dates = []
        weekdays = [5, 6]
        for dt in daterange(start_dt, end_dt):
            if dt.weekday() not in weekdays:
                dates.append(dt.strftime("%Y-%m-%d"))
        symbols.sort()

        return pd.DataFrame(index=dates, columns=symbols)

    # loop years
    more = True
    while more:

        prices = prepdf(year)
        changs = prepdf(year)
        volats = prepdf(year)

        _from = str(year)+'-01-01'
        _to = str(year)+'-12-31'
        # if it's current year
        if year == datetime.today().year:
            _to = str(datetime.today())[0:10]
            prices = prices.truncate(after=_to)
            changs = changs.truncate(after=_to)
            volats = volats.truncate(after=_to)
            more = False

        params = {'granularity': 'D', 'from': _from, 'to': _to}

        # import from oanda request (r)
        for sym in symbols:
            r = instruments.InstrumentsCandles(sym, params)
            r = api.request(r)
            for p in r['candles']:
                dtime = datetime.strftime(datetime.strptime(
                    p['time'][0:10], '%Y-%m-%d') + timedelta(days=1), '%Y-%m-%d')
                price = round((float(p['mid']['o']) + float(p['mid']['h']) + float(
                    p['mid']['l']) + float(p['mid']['c'])) / 4, len(p['mid']['c'].split('.')[1]))
                chang = round(
                    (float(p['mid']['c']) / float(p['mid']['o']) - 1) * 100, 2)
                volat = round(
                    (float(p['mid']['h']) / float(p['mid']['l']) - 1) * 100, 2)
                prices[sym][dtime] = price
                changs[sym][dtime] = chang
                volats[sym][dtime] = volat

        # clean nans
        for sym in symbols:
            changs[sym] = changs[sym].fillna(0)
            volats[sym] = volats[sym].fillna(0)
            for i in range(len(prices.index)):
                if isnan(prices[sym][i]):
                    try:
                        prices[sym][i] = prices[sym][i-1]
                    except:
                        prices[sym][i] = prices[sym][i+1]

        # create instruments db
        path = 'bin/db/instruments/'+str(year)+'/'
        Path(path).mkdir(parents=True, exist_ok=True)
        prices.to_csv(path+'prices.csv', index=True)
        changs.to_csv(path+'changs.csv', index=True)
        volats.to_csv(path+'volats.csv', index=True)

        if make_indexes:
            # make currency indices (idx)
            idx_ch = pd.DataFrame(index=prices.index, columns=ccys)
            idx_vo = pd.DataFrame(index=prices.index, columns=ccys)

            for dt in idx_ch.index:
                for ccy in idx_ch.columns:
                    n = 0
                    idx_ch[ccy][dt] = 0
                    idx_vo[ccy][dt] = 0
                    for sym in symbols:
                        if sym[0:3] == ccy:
                            idx_ch[ccy][dt] += changs[sym][dt]
                            idx_vo[ccy][dt] += volats[sym][dt]
                            n += 1
                        elif sym[4:7] == ccy:
                            idx_ch[ccy][dt] -= changs[sym][dt]
                            idx_vo[ccy][dt] += volats[sym][dt]
                            n += 1
                    idx_ch[ccy][dt] = round(idx_ch[ccy][dt] / n, 2)
                    idx_vo[ccy][dt] = round(idx_vo[ccy][dt] / n, 2)

            # create indexes db
            path = 'bin/db/indexes/'+str(year)+'/'
            Path(path).mkdir(parents=True, exist_ok=True)
            idx_ch.to_csv(path+'changs.csv', index=True)
            idx_vo.to_csv(path+'volats.csv', index=True)

            # plotting ccy indexes returns
            plt = idx_ch.plot(x=idx_ch.index, y=idx_ch.columns, height=400, title='CURRENCY INDEXES RETURNS '+str(year))
            plt.show()

            # plotting ccy indexes cumulative returns
            cum_idxs = idx_ch.cumsum()
            plt = cum_idxs.plot(x=idx_ch.index, y=idx_ch.columns, height=600, title='CURRENCY INDEXES CUMULATIVE RETURNS '+str(year))
            plt.show()

            # plotting ccy indexes volatility
            plt = idx_vo.plot.bar(x=idx_vo.index, y=idx_vo.columns, height=400, title='CURRENCY INDEXES VOLATILITY '+str(year))
            plt.show()

        print(year, 'imported!')

        year += 1


importdb( YEAR, symbols, True)
print('\ndb history updated!')


# credits: Quantium Rock


2022 imported!

db history updated!


1. PREPARING THE DATABASE WITH CURRENCY INDEXES

In order to don't charge to much our pc's RAM we'll create a local database (db) of all the symbols to be analyzed and traded. Each instrument will have it's own csv file and its timeframe folder. It will help later to backtest our model as well.

2. FILTERING NOISE WITH A LOWPASS FILTER



In [1]:
# 2. FILTERING NOISE WITH A LOWPASS FILTER

# import libraries for the model
import numpy as np
import pandas as pd
import scipy.signal as signal

# history parameters
sta = 2022 # start year
end = 2022 # end year

# importing database: indexes (idxs) and symbols (syms)
idxs = pd.read_csv('bin/db/indexes/'+str(sta)+'/changs.csv', index_col=0)
syms = pd.read_csv('bin/db/instruments/'+str(sta)+'/changs.csv', index_col=0)
for i in range(1,end-sta):
    idx2 = pd.read_csv('bin/db/indexes/'+str(sta+i)+'/changs.csv', index_col=0)
    idxs  = pd.concat([idxs, idx2])
    sym2 = pd.read_csv('bin/db/instruments/'+str(sta+i)+'/changs.csv', index_col=0)
    syms  = pd.concat([syms, sym2])

idxs = idxs.cumsum()

# lowpass filter
def lowpass( array ):
    # buterworth filter parameters
    N = 2  # filter order
    Wn = 0.07  # cutoff frequency
    B, A = signal.butter(N, Wn, output='ba')
    # apply lowpass filter
    return signal.filtfilt(B, A, array, method="gust")
    

lp = pd.DataFrame(index=idxs.index, columns=idxs.columns+'f')
lp = pd.merge(idxs, lp, left_index=True, right_index=True )
lp = lp.fillna(0)

for ccy in idxs.columns:
    lp[ccy+'f'] = lowpass(lp[ccy])


# plotting ccy indexes cumulative returns
plt = lp.plot(x=lp.index, y=lp.columns, height=600, title='CURRENCY INDEXES CUMULATIVE RETURNS + LOWPASS FILTER')
plt.show()


KeyError: "None of [Index(['2022-01-03', '2022-01-04', '2022-01-05', '2022-01-06', '2022-01-07',\n       '2022-01-10', '2022-01-11', '2022-01-12', '2022-01-13', '2022-01-14',\n       ...\n       '2022-07-07', '2022-07-08', '2022-07-11', '2022-07-12', '2022-07-13',\n       '2022-07-14', '2022-07-15', '2022-07-18', '2022-07-19', '2022-07-20'],\n      dtype='object', length=143)] are in the [columns]"

In [ ]:

import warnings
warnings.filterwarnings('ignore')
import os
import pandas as pd
pd.options.plotting.backend = "plotly"
import numpy as np
import math
import datetime as dt
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

from itertools import product
import statsmodels.api as sm

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM

from itertools import cycle
import plotly.offline as py
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

plt.style.use('seaborn-darkgrid')

: 

In [ ]:
from datetime import timedelta, date, datetime
from pathlib import Path
from numpy import isnan
import pandas as pd
pd.options.plotting.backend = "plotly"


def daterange(date1, date2):
    for n in range(int((date2 - date1).days)+1):
        yield date1 + timedelta(n)

start_dt = date(2005, 1, 1)
end_dt = date(2005, 12, 31)

dates = []
weekdays = [5, 6]
for dt in daterange(start_dt, end_dt):
    if dt.weekday() not in weekdays:
        for h in range(24):
            dtime = dt.strftime("%Y-%m-%d")
            dates.append(dtime + ' ' + str(h) + ':00:00')


print(dates)

: 

In [ ]:
# 1. PREPARING THE DATABASE WITH CURRENCY INDEXES

# import libraries for the module
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
from datetime import timedelta, date, datetime
from pathlib import Path
from numpy import isnan
import pandas as pd
pd.options.plotting.backend = "plotly"

# 
TOKEN = "37d11dc6e01ec900277bcf70ed296912-60827d532c22d5b1b0d3583a0e9309ef" # YOUR OANDA TOKEN
#
YEAR = 2020 # YEARS TO START
TIMEFRAME = 'M1' # ONE MINUTE
#


# instruments universe
symbols = ['AUD_CAD', 'AUD_CHF', 'AUD_HKD', 'AUD_JPY', 'AUD_NZD', 'AUD_SGD', 'AUD_USD',
           'CAD_CHF', 'CAD_HKD', 'CAD_JPY', 'CAD_SGD', 'CHF_HKD', 'CHF_JPY', 'EUR_AUD',
           'EUR_CAD', 'EUR_CHF', 'EUR_GBP', 'EUR_HKD', 'EUR_JPY', 'EUR_NZD', 'EUR_SGD',
           'EUR_USD', 'GBP_AUD', 'GBP_CAD', 'GBP_CHF', 'GBP_HKD', 'GBP_JPY', 'GBP_NZD',
           'GBP_SGD', 'GBP_USD', 'HKD_JPY', 'NZD_CAD', 'NZD_CHF', 'NZD_HKD', 'NZD_JPY',
           'NZD_SGD', 'NZD_USD', 'SGD_CHF', 'SGD_HKD', 'SGD_JPY', 'USD_CAD', 'USD_CHF',
           'USD_HKD', 'USD_JPY', 'USD_SGD']  # 45 FX pairs


api = API(access_token=TOKEN)

# extract currencies (ccys)
ccys = []
for sym in symbols:
    p = sym.split('_')
    if len(p[0]) == 3 and p[0] not in ccys:
        ccys.append(p[0])
    if len(p[1]) == 3 and p[1] not in ccys:
        ccys.append(p[1])
    ccys.sort()

# loop years
year = YEAR
more = True
while more:

    prices = pd.DataFrame()
    changs = pd.DataFrame()
    volats = pd.DataFrame()

    _from = str(year)+'-01-01'
    _to = str(year)+'-12-31'
    # if it's current year
    if year == datetime.today().year:
        _to = str(datetime.today())[0:19]
        print(_to)
        prices = prices.truncate(after=_to)
        changs = changs.truncate(after=_to)
        volats = volats.truncate(after=_to)
        more = False

    params = {'granularity': 'M1', 'from': _from, 'to': _to}

    # import from oanda request (r)
    for sym in symbols:
        r = instruments.InstrumentsCandles(sym, params)
        r = api.request(r)
        for p in r['candles']:
            dtime = datetime.strftime(datetime.strptime(
                p['time'][0:10], '%Y-%m-%d') + timedelta(days=1), '%Y-%m-%d')
            price = round((float(p['mid']['o']) + float(p['mid']['h']) + float(
                p['mid']['l']) + float(p['mid']['c'])) / 4, len(p['mid']['c'].split('.')[1]))
            chang = round(
                (float(p['mid']['c']) / float(p['mid']['o']) - 1) * 100, 2)
            volat = round(
                (float(p['mid']['h']) / float(p['mid']['l']) - 1) * 100, 2)
            prices[sym][dtime] = price
            changs[sym][dtime] = chang
            volats[sym][dtime] = volat

    # clean nans
    for sym in symbols:
        changs[sym] = changs[sym].fillna(0)
        volats[sym] = volats[sym].fillna(0)
        for i in range(len(prices.index)):
            if isnan(prices[sym][i]):
                try:
                    prices[sym][i] = prices[sym][i-1]
                except:
                    prices[sym][i] = prices[sym][i+1]

    # create instruments db
    path = 'bin/db/instruments/'+str(year)+'/'
    Path(path).mkdir(parents=True, exist_ok=True)
    prices.to_csv(path+'prices.csv', index=True)
    changs.to_csv(path+'changs.csv', index=True)
    volats.to_csv(path+'volats.csv', index=True)

    # make currency indices (idx)
    idx_ch = pd.DataFrame(index=prices.index, columns=ccys)
    idx_vo = pd.DataFrame(index=prices.index, columns=ccys)

    for dt in idx_ch.index:
        for ccy in idx_ch.columns:
            n = 0
            idx_ch[ccy][dt] = 0
            idx_vo[ccy][dt] = 0
            for sym in symbols:
                if sym[0:3] == ccy:
                    idx_ch[ccy][dt] += changs[sym][dt]
                    idx_vo[ccy][dt] += volats[sym][dt]
                    n += 1
                elif sym[4:7] == ccy:
                    idx_ch[ccy][dt] -= changs[sym][dt]
                    idx_vo[ccy][dt] += volats[sym][dt]
                    n += 1
            idx_ch[ccy][dt] = round(idx_ch[ccy][dt] / n, 2)
            idx_vo[ccy][dt] = round(idx_vo[ccy][dt] / n, 2)

    # create indexes db
    path = 'bin/db/indexes/'+str(year)+'/'
    Path(path).mkdir(parents=True, exist_ok=True)
    idx_ch.to_csv(path+'changs.csv', index=True)
    idx_vo.to_csv(path+'volats.csv', index=True)

    # plotting ccy indexes returns
    plt = idx_ch.plot(x=idx_ch.index, y=idx_ch.columns, height=400, title='CURRENCY INDEXES RETURNS '+str(year))
    plt.show()

    # plotting ccy indexes cumulative returns
    cum_idxs = idx_ch.cumsum()
    plt = cum_idxs.plot(x=idx_ch.index, y=idx_ch.columns, height=600, title='CURRENCY INDEXES CUMULATIVE RETURNS '+str(year))
    plt.show()

    # plotting ccy indexes volatility
    plt = idx_vo.plot.bar(x=idx_vo.index, y=idx_vo.columns, height=400, title='CURRENCY INDEXES VOLATILITY '+str(year))
    plt.show()

    print(year, 'imported!')

    year += 1


print('\ndb history updated!')


# credits: Quantium Rock


: 

: 